#**Домашнее задание №3: RAG**


*В рамках задания была реализована RAG-система из датасета bearberry/rus_xquadqa.*

Основные этапы работы включали:

1.   Предобработку данных с удалением дубликатов и разбиением текстов на чанки.
2.   Создание гибридной поисковой системы (векторный поиск + BM25).
3.   Настройку двух моделей для генерации ответов.
4.   Сравнение качества ответов моделей на топ-10 вопросов.

###Код
Для начала подготавливаем окружение:

In [ ]:
# установка и импорт необходимых библиотек
!pip install datasets qdrant-client transformers sentence-transformers
!pip install -U langchain-community

from datasets import load_dataset
from qdrant_client import QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Загружаем датасет и подготавливаем корпус:**

Исходный датасет содержал 1190 контекстов, из которых после удаления точных дубликатов осталось 241 уникальных.

In [ ]:
# загрузка датасета и подготовка корпуса, а именно объединение фрагментов из поля 'context' и удаление дублей

dataset = load_dataset("bearberry/rus_xquadqa", split="train")

# объединяем фрагменты, удаляем точные дубликаты
all_contexts = [
    " ".join([chunk["chunk"] for chunk in item["context"]]) if isinstance(item["context"], list) else item["context"]
    for item in dataset
]
unique_contexts = list(set(all_contexts))

print(f"Всего контекстов: {len(all_contexts)}")
print(f"Всего уникальных контекстов: {len(unique_contexts)}")

# ПРИМЕР: выводиим первые 2 уникальные контекста
print("\n\nПримеры уникальных контекстов:")
for ctx in unique_contexts[:2]:
    print("------------------------------")
    print(ctx)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

rus_xquadqa.json:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1190 [00:00<?, ? examples/s]

Всего контекстов: 1190
Всего уникальных контекстов: 241


Примеры уникальных контекстов:
------------------------------
Первым описанным поселением на месте современного Ньюкасла был Понс-Элиус, римский форт и мост через реку Тайн. Ему дали фамилию римского императора Адриана, который основал его во 2 веке н. э. Эта редкая честь позволяет предположить, что Адриан, возможно, посетил это место и построил мост во время своего путешествия по Британии. Население Понс-Элиуса в это время составляло примерно 2 000 человек. Фрагменты Вала Адриана до сих пор можно увидеть в некоторых районах Ньюкасла, особенно вдоль Вест Роуд. "Римскую стену" можно проследить до римского форта Сегедунум в Уолсэенде — "конца стены" (англ. "wall's end") — и до форта снабжения Арбея в Саут-Шилдс. Протяженность стены Адриана составляла 73 мили (117 км), охватывая всю ширину Британии; стена включала Валлум, большой задний ров с параллельными насыпями, и была сооружена в первую очередь для защиты, предотвращения нежел

**Разбиение фрагментов на чанки:**

Длинные контексты были разбиты на чанки по 3 предложения с перекрытием в 1 предложение, что должно позволить сохранить смысловую целостность и увеличить релевантность. Всего получился 571 чанк.

In [ ]:
# установка и импорт необходимых библиотек для разделения фрагментов на чанки
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from tqdm import tqdm   # для отслеживания процесса (в целом, необязательно)
nltk.download('punkt_tab')

# функция дележки на чанки
def split_text_into_chunks(text, chunk_size=3, overlap_size=1): #экспериментально подобрал так
    """
    Разбиение текста на чанки по предложениям с перекрытием.
    :param text: текст для разбиения.
    :param chunk_size: максимальное количество предложений в одном чанке.
    :param overlap_size: количество предложений для перекрытия между чанками.
    :return: список чанков.
    """
    sentences = sent_tokenize(text)  # разбиваем текст на предложения
    chunks = []

    start = 0
    while start < len(sentences):
        end = min(start + chunk_size, len(sentences))  # конец чанка
        chunk = " ".join(sentences[start:end])  # формируем чанк
        chunks.append(chunk)

        if end == len(sentences):  # выходим, если достигли конца текста
            break

        start += chunk_size - overlap_size  # смещаем с учетом перекрытия

    return chunks


# разбиваем уже наши данные
final_corpus = []
print("Разбиваем наши тексты на чанки....")
for ctx in tqdm(unique_contexts, desc="ЧАНКИНГ", unit="текст"):
    # проверка длины контекста, чтобы не разбивать слишком короткие
    if len(ctx) > 500:
        final_corpus.extend(split_text_into_chunks(ctx))
    else:
        final_corpus.append(ctx)

# Вывод результатов
print(f"\n\nРАЗБИЕНИЕ ЗАВЕРШЕНО: всего чанков в финальном корпусе: {len(final_corpus)}")
print("\n\nПримеры чанков:")
for chunk in final_corpus[:5]:
    print("-----")
    print(chunk)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Разбиваем наши тексты на чанки....


ЧАНКИНГ: 100%|██████████| 241/241 [00:00<00:00, 2557.12текст/s]



РАЗБИЕНИЕ ЗАВЕРШЕНО: всего чанков в финальном корпусе: 571


Примеры чанков:
-----
Первым описанным поселением на месте современного Ньюкасла был Понс-Элиус, римский форт и мост через реку Тайн. Ему дали фамилию римского императора Адриана, который основал его во 2 веке н. э. Эта редкая честь позволяет предположить, что Адриан, возможно, посетил это место и построил мост во время своего путешествия по Британии. Население Понс-Элиуса в это время составляло примерно 2 000 человек.
-----
Население Понс-Элиуса в это время составляло примерно 2 000 человек. Фрагменты Вала Адриана до сих пор можно увидеть в некоторых районах Ньюкасла, особенно вдоль Вест Роуд. "Римскую стену" можно проследить до римского форта Сегедунум в Уолсэенде — "конца стены" (англ.
-----
"Римскую стену" можно проследить до римского форта Сегедунум в Уолсэенде — "конца стены" (англ. "wall's end") — и до форта снабжения Арбея в Саут-Шилдс. Протяженность стены Адриана составляла 73 мили (117 км), охватывая всю ширину Бр

**Реализация гибридного поиска:**

Для повышения точности поиска реализована комбинация 2-х методов, а именно:

*   векторный поиск через Qdrant (модель: paraphrase-multilingual-MiniLM-L12-v2),
*   BM25 - для поиска по ключевым словам.

Этот подход позволяет находить как семантически близкие, так и лексически релевантные ответы.

In [ ]:
# установка и импорт необходимых библиотек для создания векторной БД
!pip install rank_bm25 langchain
import re
from rank_bm25 import BM25Okapi
from qdrant_client import QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant

# используем модель эмбеддингов, которая точно подойдет для русского языка
qdrant_client = QdrantClient(":memory:")
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Qdrant.from_texts(
    texts=final_corpus,
    embedding=embedding_function,
    location=":memory:",
    collection_name="rus_xquadqa_collection",
    url=None
)

# построение BM25-индекса по final_corpus
def preprocess(text):
    """Приводит текст к нижнему регистру и извлекает слова"""
    tokens = re.findall(r'\w+', text.lower())
    return tokens
tokenized_corpus = [preprocess(doc) for doc in final_corpus]
bm25 = BM25Okapi(tokenized_corpus)


# реализация гибридного поиска
def hybrid_search(query, vectorstore, bm25, final_corpus, top_k=5, weight_vector=0.5, weight_bm25=0.5):
    """
    Гибридный поиск, комбинирующий векторный поиск и BM25.
    :param query: запрос пользователя.
    :param vectorstore: объект векторного хранилища.
    :param bm25: BM25 индекс, построенный на final_corpus.
    :param final_corpus: список текстов, по которым построен BM25 индекс.
    :param top_k: число возвращаемых чанков.
    :param weight_vector: весовой коэффициент для векторного поиска.
    :param weight_bm25: весовой коэффициент для BM25.
    :return: список из top-k лучших результатов.
    """
    # препроцессинг запроса для BM25
    tokenized_query = preprocess(query)
    bm25_scores = bm25.get_scores(tokenized_query)  # получаем BM25-скор для каждого документа

    # получаем кандидатов и запрашиваем больше кандидатов для объединения
    vector_results = vectorstore.similarity_search(query, k=top_k*3)

    combined = []
    for result in vector_results:
        text = result.page_content
        try:
            idx = final_corpus.index(text)
        except ValueError:
            continue

        score_bm25 = bm25_scores[idx]
        score_vector = result.score if hasattr(result, "score") else 0.0

        combined_score = weight_vector * score_vector + weight_bm25 * score_bm25
        combined.append((combined_score, result))

    combined.sort(key=lambda x: x[0], reverse=True) # сортировка кандидатов

    best_results = [res for score, res in combined][:top_k]
    return best_results

# пример
print("\n\nВыполняем гибридный поиск для запроса:")
query = "Сколько очков набрал игрок?"
print(query)
hybrid_results = hybrid_search(query, vectorstore, bm25, final_corpus, top_k=3, weight_vector=0.5, weight_bm25=0.5)
print("\nРезультаты поиска:")
for res in hybrid_results:
    print("--------------")
    print(res.page_content)


<ipython-input-4-1d000a052c87>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]



Выполняем гибридный поиск для запроса:
Сколько очков набрал игрок?

Результаты поиска:
--------------
Защита Пэнтерс уступила всего 308 очков, заняв шестое место в лиге, а также лидировала в НФЛ по перехватам с 24 и похвасталась четырьмя попаданиями в Пробоул. Дифенсив тэкл Пробоула Кейван Шорт лидирует в команде с 11 мешками, а также обеспечил три потери мяча и получил два. Нападающий Марио Эдисон добавил 61⁄2 мешков.
--------------
Бронкос победил Питтсбург Стилерс в дивизионном раунде, 23–16, набрав 11 очков в последние три минуты игры. Затем они победили действующего чемпиона Суперкубка XLIX Нью-Ингленд Пэтриотс в игре чемпионата АФК, 20–18, перехватив пас на попытку двухочковой конверсии Нью-Ингленд, когда на часах оставалось 17 секунд. Несмотря на проблемы Мэннинга с перехватами в течение сезона, он не проиграл ни одной в их двух игр плей-офф.
--------------
Позади них для участия в Пробоуле также были выбраны два из трех стартовых лайнбекеров Пэнтерс: Томас Дэвис и Люк Кикли. 

**Настройка моделей:**

Для генерации ответов выбраны следующие модели:

1) Microsoft/Phi-4-mini-instruct - модель на 4B.

2) Qwen/Qwen2.5-1.5B-Instruct - модель на 1.5B.

Обе модели успешно интегрированы в RAG-цепочку. Для лучшей генерации было выбрано малое число токенов и маленькая температура.
В качестве примера выведены ответы от каждой из моделей на 1-й вопрос из датасета, а также результаты поиска по БД.

In [ ]:
# установка и импорт необходимых библиотек
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

#  подготовка для подачи в модель
def apply_chat(messages, tokenizer):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

# параметры генерации
pipeline_kwargs = {"max_new_tokens": 64, "do_sample": True, "temperature": 0.05}

# МОДЕЛЬ 1: microsoft/Phi-4-mini-instruct
model_name_phi = 'microsoft/Phi-4-mini-instruct'
model_phi = AutoModelForCausalLM.from_pretrained(
    model_name_phi,
    torch_dtype=torch.float16,
    device_map='auto'
)
model_phi.eval()
tokenizer_phi = AutoTokenizer.from_pretrained(model_name_phi)
pipe_phi = pipeline("text-generation", model=model_phi, tokenizer=tokenizer_phi, **pipeline_kwargs)
hugging_face_pipeline_phi = HuggingFacePipeline(pipeline=pipe_phi)

# МОДЕЛЬ 2: Qwen/Qwen2.5-1.5B-Instruct
model_name_qwen = 'Qwen/Qwen2.5-1.5B-Instruct'
model_qwen = AutoModelForCausalLM.from_pretrained(
    model_name_qwen,
    torch_dtype=torch.float16,
    device_map='auto'
)
model_qwen.eval()
tokenizer_qwen = AutoTokenizer.from_pretrained(model_name_qwen)
pipe_qwen = pipeline("text-generation", model=model_qwen, tokenizer=tokenizer_qwen, **pipeline_kwargs)
hugging_face_pipeline_qwen = HuggingFacePipeline(pipeline=pipe_qwen)


# ПРИМЕР для первого вопроса из нашего датасета
query = 'Сколько очков уступила защита Пэнтерс?'
# получаем релевантные документы с помощью retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
relevant_docs = retriever.get_relevant_documents(query)
docs_text = '\n\n'.join([doc.page_content for doc in relevant_docs])

chat = [
    {"role": "user", "content": f"Релевантная информация: {docs_text}\n\nВопрос: {query}"},
]

# генерация моделью 1
prompt_phi = apply_chat(chat, tokenizer_phi)
res_phi = hugging_face_pipeline_phi(prompt_phi)
print("Ответ модели microsoft/Phi-4-mini-instruct:")
print(res_phi)

# генерация моделью 2
prompt_qwen = apply_chat(chat, tokenizer_qwen)
res_qwen = hugging_face_pipeline_qwen(prompt_qwen)
print("\nОтвет модели Qwen/Qwen2.5-1.5B-Instruct:")
print(res_qwen)

# гибридный поиск (для проверки)
print("\n\n\nРезультаты гибридного поиска:")
hybrid_results = hybrid_search(query, vectorstore, bm25, final_corpus, top_k=3, weight_vector=0.5, weight_bm25=0.5)
for res in hybrid_results:
    print("-----")
    print(res.page_content)

config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-5-929b72ef1e3d>:43: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)
<ipython-input-5-929b72ef1e3d>:52: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res_phi = hugging_face_pipeline_phi(prompt_phi)


Ответ модели microsoft/Phi-4-mini-instruct:
<|user|>Релевантная информация: Защита Пэнтерс уступила всего 308 очков, заняв шестое место в лиге, а также лидировала в НФЛ по перехватам с 24 и похвасталась четырьмя попаданиями в Пробоул. Дифенсив тэкл Пробоула Кейван Шорт лидирует в команде с 11 мешками, а также обеспечил три потери мяча и получил два. Нападающий Марио Эдисон добавил 61⁄2 мешков.

Нападающий Марио Эдисон добавил 61⁄2 мешков. Линия Пэнтерс также представила ди-энда-ветерана Джареда Аллена, пятикратного участника Пробоула, который был активным лидером по количеству мешков в карьере НФЛ в количестве 136, вместе с ди-эндом Кони Или, у которого было 5 мешков всего за 9 стартов. Позади них для участия в Пробоуле также были выбраны два из трех стартовых лайнбекеров Пэнтерс: Томас Дэвис и Люк Кикли.

Позади них для участия в Пробоуле также были выбраны два из трех стартовых лайнбекеров Пэнтерс: Томас Дэвис и Люк Кикли. Дэвис собрал 51⁄2 мешков, четыре вынужденных потери мяча и че

**Генерация ответов на топ-10 вопросов моделями:**

In [ ]:
# выводим топ-10 вопросов и ответов на них с помощью наших двух моделей

questions = []
gold_answers = []

for i in range(10):
    item = dataset[i]
    questions.append(item["question"])
    gold_answers.append(item["answers"][0] if item["answers"] else "") #берем первый вариант из голда

for idx, (q, gold) in enumerate(zip(questions, gold_answers)):
    print(f"{idx+1}. Q: {q}\n   A (gold): {gold}\n")
    relevant_docs = retriever.get_relevant_documents(q)
    docs_text = '\n\n'.join([doc.page_content for doc in relevant_docs])

    chat = [
        {"role": "user", "content": f"Релевантная информация: {docs_text}\n\nВопрос: {q}"},
    ]

# генерация ответов
    phi_response = hugging_face_pipeline_phi(apply_chat(chat, tokenizer_phi))
    phi_answer = phi_response.split("<|assistant|>")[1].strip() if "<|assistant|>" in phi_response else phi_response.strip()
    print(f"   Ответ модели microsoft/Phi-4-mini-instruct: {phi_answer}\n")

    qwen_response = hugging_face_pipeline_qwen(apply_chat(chat, tokenizer_qwen))
    qwen_answer = qwen_response.split("<|im_start|>assistant")[1].strip() if "<|im_start|>assistant" in qwen_response else qwen_response.strip()
    print(f"   Ответ модели Qwen/Qwen2.5-1.5B-Instruct: {qwen_answer}\n")

1. Q: Сколько очков уступила защита Пэнтерс?
   A (gold): 308

   Ответ модели microsoft/Phi-4-mini-instruct: Защита Пэнтерс уступила всего 308 очков.

   Ответ модели Qwen/Qwen2.5-1.5B-Instruct: Защита Пэнтерс уступила всего 308 очков.

2. Q: Сколько мешков за карьеру было у Джареда Аллена?
   A (gold): 136

   Ответ модели microsoft/Phi-4-mini-instruct: У Джареда Аллена было 136 мешков за карьеру.

   Ответ модели Qwen/Qwen2.5-1.5B-Instruct: Джаред Аллен играл в НФЛ 136 матчей, что составляет 136 мешков за свою карьеру.

3. Q: Сколько блокировок записал на свой счет Люк Кикли?
   A (gold): 118

   Ответ модели microsoft/Phi-4-mini-instruct: Люк Кикли записал 118 блокировок на свой счет.

   Ответ модели Qwen/Qwen2.5-1.5B-Instruct: По информации предоставленной в вопросе, Люк Кикли лидировал в команде по блокировкам (118) во время Пробоула.

4. Q: Сколько мячей перехватил Джош Норман?
   A (gold): 4

   Ответ модели microsoft/Phi-4-mini-instruct: Джош Норман перехватил 4 мяча.

   Отв

##Анализ результатов:

**Пройдемся по результатам для каждого из 10 вопросов:**
1.   На этот вопрос обе модели дали правильный ответ.
2.   На этот вопрос обе модели также дали правильный ответ, но модель от Qwen расписала ответ более подробно.
3.   На этот вопрос обе модели также дали правильный ответ, но модель от Qwen ответила более расплывчато и нечетко.
4.   На этот вопрос обе модели также дали правильный ответ, но модель от Qwen ответила чуть более подробно, добавив фразу "во время этой игры", что не очень корректно.
5.   Обе модели дали неверный ответ на этот вопрос.
6.   На этот вопрос обе модели дали правильный ответ.
7.   Обе модели дали неверный ответ на этот вопрос.
8.   Обе модели дали неверный ответ на этот вопрос.
9.   Модель от Microsoft дала правильный ответ, модель от Qwen ответила неверно.
10.  Аналогично - модель от Microsoft дала правильный ответ, модель от Qwen ответила неверно, причем обе модели ответили не очень кратко.


##Вывод

####Точность моделей:

1.   microsoft/Phi-4-mini-instruct — 7/10 правильных ответов (лучше в конкретных фактах).
2.   Qwen/Qwen2.5-1.5B-Instruct — 5/10 (больше "словоблудит", возможно, из-за "неуверенности" в ответе, что может быть связано с более низким числом параметров).

Несмотря на положительные результаты, есть вопросы, на которые обе модели не смогли дать верный ответ, возможно, из-за сложности или недостатка данных.

Так, RAG-система была успешно реализована.